### Statistiques concernant les cararactéristiques des incidents ayant fait des victimes : comment expliquer le nombre de victimes en fonction de l'incident ?

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from plotly.offline import init_notebook_mode
init_notebook_mode(connected= True)
import plotly.express as px

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
url="https://drive.google.com/file/d/1LO4i_-TkSmxnR0dgO5_lVLZ35FtGTNzA/view?usp=drive_link"
url="https://drive.google.com/uc?export=download&confirm=1&id=" + url.split("/")[-2]
total_db = pd.read_csv(url)
total_db.sample(5)

In [ ]:
gun_violence_db = pd.read_csv('data/gun_violence_db.csv')
gun_violence_db.sample(5)

In [ ]:
from shapely import wkt
def wkt_loads(x):
    try:
        return wkt.loads(x)
    except Exception:
        return None
total_db = total_db.dropna(subset=['geometry'])
total_db['geometry'] = total_db['geometry'].apply(wkt_loads)

In [ ]:
total_db = total_db.loc[total_db['Etat'] != 'Alaska']
total_db = total_db.loc[total_db['Etat'] != 'Hawaii']

gun_violence_db = gun_violence_db.loc[gun_violence_db['state'] != 'Alaska']
gun_violence_db = gun_violence_db.loc[gun_violence_db['state'] != 'Hawaii']


In [ ]:
total_db_geo = (
    gpd.GeoDataFrame(total_db)
    .set_geometry('geometry')
    .set_crs('EPSG:4269')
)

In [ ]:
#changement de valeur pour Richmond, Virginia car valeur aberrante de 8.755 alors qu'il y avait 204 214	habitants en 2010
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2013'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2014'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2015'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2016'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2017'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2018'] = 204.214

In [ ]:
total_pop=total_db_geo.groupby('Etat').agg({'resident_pop_year_2015':'sum'}).reset_index()
total_pop.rename(columns={'resident_pop_year_2015':'pop_state_2015'}, inplace=True)
total_pop

In [ ]:
total_db_geo=total_db_geo.merge(total_pop, on ='Etat', how='left')

In [ ]:
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(gun_violence_db['longitude'], gun_violence_db['latitude'])]
gun_violence_geo = gpd.GeoDataFrame(gun_violence_db, geometry=geometry, crs='EPSG:4326')

gun_violence_geo = gun_violence_geo.set_crs('EPSG:4326')
gun_violence_geo = gun_violence_geo[gun_violence_geo.geometry.x <= -50]

In [ ]:
total_db_geo=total_db_geo.to_crs('EPSG:4326')

In [ ]:
merge_geo = gpd.sjoin(gun_violence_geo, total_db_geo, predicate='within')
merge_geo.to_csv('rendu_final/data/merge_geo.csv')

In [ ]:
geom_county=total_db_geo[['id_county','geometry']]
geom_county.head()

In [ ]:
merge_geo2=pd.merge(merge_geo, geom_county, on='id_county')
merge_geo2.head()

In [ ]:
merge_geo = merge_geo2.rename(columns={'geometry_x': 'point_geometry', 'geometry_y': 'polygon_geometry'})
merge_geo.head()

In [ ]:
base_totale=merge_geo.copy()

## Après cet import et traitement rapide des données, on peut passer aux statistiques

In [ ]:
print("I. Les dates des incidents peuvent-elles avoir un effet sur les conséquences de celui-ci ? \n \n Nous allons commencer cette analyse des conséquences des incidents en termes de victimes (blessés et morts) par l'étude des effets potentiels de la date sur le nombre et la gravité des incidents.")

In [ ]:
base_totale['date']=pd.to_datetime(base_totale['date'])

In [ ]:
grouped_data = base_totale.groupby(base_totale['date'].dt.year).agg({
    'n_injured': 'sum',
    'n_killed': 'sum'
}).reset_index()

fig1 = px.bar(
    grouped_data,
    x=grouped_data['date'],
    y=['n_injured', 'n_killed'],
    labels={"date": "année", "value": "Nombre de victimes", "variable": "Type de victime"},
    title="Nombre de victimes d'incidents par arme à feu par an",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig1.update_layout(barmode='group')  # Display bars side by side

fig1.show()


In [ ]:
print("Ce graphique nous montre que, non seulement le nombre d'incidents par arme à feu augmente sur nos 4 années complètes considérées, mais que le nombre de victimes (blessés et tués) augmente également.\nAinsi, les conséquences de ces incidents touchent de plus en plus d'individus au fur et à mesure des années.")

In [ ]:
grouped_data2 = base_totale.groupby(base_totale['date'].dt.year).agg({
    'n_injured': 'mean',
    'n_killed': 'mean'
}).reset_index()

grouped_data2=grouped_data2.loc[grouped_data2['date'].isin([2014,2015,2016,2017])]

fig2 = px.bar(
    grouped_data2,
    x=grouped_data2['date'],
    y=['n_injured', 'n_killed'],
    labels={"date": "mois", "value": "Moyenne de victimes par incident", "variable": "Type de victime"},
    title="Nombre moyen de victimes par incident par arme à feu par an",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig2.update_layout(barmode='group')  # Display bars side by side

fig2.show()

In [ ]:
print("Ce second graphique nous montre que l'augmentation des victimes repérée précédemment est liée quasiment exclusivement à l'augementation du nombre d'incident, car le nombre de victime par incident reste relativement stable sur les 4 années observées, mis à part une légèrement augementation du nombre de blessés moyen par incident entre 2014 et 2015.")

In [ ]:
grouped_data3 = base_totale.loc[base_totale['date'].dt.year.isin([2014,2015,2016,2017])].groupby(
    base_totale['date'].dt.month).agg({
    'n_injured': 'sum',
    'n_killed': 'sum'
}).reset_index()


fig3 = px.bar(
    grouped_data3,
    x=grouped_data3['date'],
    y=['n_injured', 'n_killed'],
    labels={"date": "Mois", "value": "Nombre de victimes", "variable": "Type de victime"},
    title="Nombre de victimes d'incidents par arme à feu par mois (cumulé de 2014 à 2017)",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig3.update_layout(barmode='group')  # Display bars side by side

fig3.show()



In [ ]:
grouped_data4 = base_totale.loc[base_totale['date'].dt.year.isin([2014,2015,2016,2017])].groupby(
    base_totale['date'].dt.month).agg({
    'n_injured': 'mean',
    'n_killed': 'mean'
}).reset_index()


fig4 = px.bar(
    grouped_data4,
    x=grouped_data4['date'],
    y=['n_injured', 'n_killed'],
    labels={"date": "année", "value": "Nombre moyen de victimes par incident", "variable": "Type de victime"},
    title="Moyenne de victimes d'incident par arme à feu par mois (entre 2014 et 2017)",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig4.update_layout(barmode='group')  # Display bars side by side

fig4.show()

In [ ]:
print("On retrouve ici la même variation que pour le nombre d'incidents : il y a plus de blessés et de morts par arme à feu pendant l'été aux Etats-Unis que pendant l'hiver. \nDe plus, même si le nombre de morts par incident reste stable autour de 0,25, le nombre moyen de victimes par incident varie de 0.46 en Février à 0.55 en Juillet. \nIl semble donc exister un effet de la saisonnalité sur le nombre d'incidents mais également sur leur gravité et leurs conséquences en termes de victimes.")

In [ ]:
print("\n II. Le type d'armes utilisé influe-t-il le nombre de victimes lors d'un incident ? \n")

In [ ]:
grouped_data5 = base_totale.groupby('gun_type').agg({
    'n_injured': 'sum',
    'n_killed': 'sum'
}).reset_index()


fig5 = px.bar(
    grouped_data5,
    x=grouped_data5['gun_type'],
    y=['n_injured', 'n_killed'],
    labels={"gun_type": "Arme", "value": "Nombre de victimes", "variable": "Type de victime"},
    title="Nombre de victimes en fonction de l'arme à feu",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig5.update_layout(barmode='group')  # Display bars side by side

fig5.show()

In [ ]:
print("Ce graphique, en échelle logarithmique pour la lisibilité, nous montre que, de manière générale, les armes utilisées font un nombre similaire de blessés et de morts. Il n'y a donc pas d'arme qui ne fasse que des blessés mais pas de mort et inversement. \n De plus,les quatre types d'armes faisant le plus de victimes sont les mêmes que celles présentes dans le plus d'incidents (Handgun, Rifle, Shotgun, 9mm).")

In [ ]:
grouped_data6 = base_totale.groupby('gun_type').agg({
    'n_injured': 'mean',
    'n_killed': 'mean'
}).reset_index()


fig6 = px.bar(
    grouped_data6,
    x=grouped_data6['gun_type'],
    y=['n_injured', 'n_killed'],
    labels={"gun_type": "Arme", "value": "Nombre moyen de victimes par incident", "variable": "Type de victime"},
    title="Nombre moyen de victimes par incident en fonction de l'arme à feu",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig6.update_layout(barmode='group')  # Display bars side by side

fig6.show()

In [ ]:
print("Concernant les types d'armes qui étaient significatives en termes de nombre de victimes, on remarque que celle qui fait le plus de blessés et de morts en moyenne est la carabine ('Rifle'), suivie du fusil de chasse ('Shotgun'). \n Ces deux armes sont des armes militaires ou de chasse, elles sont donc probablement possédées par des personnes plus à l'aise avec les armes ou plus puissantes, ce qui pourrait expliquer qu'elles fassent plus de victimes. \n L'arme de poing fait mois de blessés et morts en moyenne que ces deux armes précédentes et le '9mm' fait particulièrement peu de victimes en moyenne en comparaison avec les autres. \n Il ne faut pas oublier la catégorie 'arme inconnue' qui fait plus de victimes en moyenne que la carabine et qui représente un échantillon bien plus grand.")

In [ ]:
grouped_data7 = base_totale.groupby('young_sus').agg({
    'n_injured': 'mean',
    'n_killed': 'mean',
    'pop_state_2015':'first'
}).reset_index()

fig7 = px.bar(
    grouped_data7,
    x=grouped_data7['young_sus'],
    y=['n_injured', 'n_killed'],
    labels={"young_sus": "Suspect entre 18 et 25 ans (1)", "value": "Moyenne du nombre de victimes", "variable": "Type de victime"},
    title="Nombre moyen de victimes en fonction de l'âge du suspect",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig7.update_layout(barmode='group')  # Display bars side by side

fig7.show()

In [ ]:
print("Lorsque le suspect est jeune, entre 18 et 25 ans, il provoque en moyenne plus de morts lors d'un incident par arme à feu que lorsqu'il appartient à une autre catégorie d'âge (0.28 contre 0.24). \n Néanmoins, un suspect entre 18 et 25 ans fait, en moyenne, moins de blessés qu'un autre type de suspect (0.38 contre 0.53).")

In [ ]:
print(" \n IV. Les circonstances de l'incident par arme à feu influencent-elles ses conséquences ? \n ")

In [ ]:
grouped_data8 = base_totale.groupby('relation').agg({
    'n_injured': 'mean',
    'n_killed':'mean'
}).reset_index()


fig8 = px.bar(
    grouped_data8,
    x=grouped_data8['relation'],
    y=['n_injured', 'n_killed'],
    labels={"relation": "Relation entre suspect(s) et victime(s)", "value": "Moyenne du nombre de victimes", "variable": "Type de victime"},
    title="Nombre moyen de victimes en fonction de la relation du suspect aux victimes",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig8.update_layout(barmode='group')  # Display bars side by side

fig8.show()

In [ ]:
print("Ce graphique mets en évidence de fortes disparités de conséquences selon les différents types d'incidents par arme à feu, ici caractérisés par les différentes relation du suspect aux victimes. \nEn effet, on remarque que le type d'incident provoquant, en moyenne, le plus de blessés et de morts est la tuerie de masse. Lorsque le suspect connaît les victimes, celles-ci ont plus de chances de mourir que d'être blessées (en moyenne presque 3 morts mais 1 blessé) alors que quand les victimes sont inconnues, celles-ci ont plus de chances d'être blessées que tuées (en moyenne 4 blessés mais 1.4 morts). \nLe deuxième type d'incident par arme à feu provoquant le plus de morts sont les incidents dans le cadre familial. Lorsque la victime et un(e) (ex-)partnaire du suspect, il y a en moyenne 0.7 morts (et 0.3 blessés) et lorsqu'elle fait partie de la famille du suspect, il y a en moyenne 0.8 morts (et 0.4 blessés). \nAinsi, les violences conjugales et intrafamiliales font partie des violences par arme à feu les plus meurtrières, avant les violences entre gangs, par exemple. \nAprès les tueries de masse, les incidents par arme à feu provoquant le plus de blessés sont les violences de gangs (0.8 blessés et 0.47 morts en moyenne) puis les vols à main armée (0.7 blessés et 0.2 morts en moyenne).")

In [ ]:
grouped_data9 = base_totale.groupby('relation').agg({
    'n_injured': 'sum',
    'n_killed': 'sum'
}).reset_index()


fig9 = px.bar(
    grouped_data9,
    x=grouped_data9['relation'],
    y=['n_injured', 'n_killed'],
    labels={"relation": "Relation entre suspect(s) et victime(s)", "value": "nombre de victimes", "variable": "Type de victime"},
    title="Nombre de victimes en fonction de la relation du suspect aux victimes",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"}
)

fig9.update_layout(barmode='group')  # Display bars side by side

fig9.show()

In [ ]:
print(f"Ce graphique reprenant la même classification des types d'incidents par arme à feu que le précédent permet de se rendre compte des conséquences de ces incidents à l'échelle agrégée plutôt qu'en moyenne. \nLe type d'incidents provoquant le plus de blessés est alors le vol à main armé ({3535/grouped_data9['n_injured'].sum()*100}% des blessés dont la relation avec le suspect est précisée), notamment en raison de leur fréquence très élevée. \nLes violences conjugales et intrafamiliales sont le type d'incident par arme à feu le plus meurtrier ({(3009+2556)/grouped_data9['n_killed'].sum()*100}% des morts dont on connaît la relation au suspect).")

In [ ]:
print("\nV. Les conséquences d'un incident dépendent-elles de l'Etat dans lequel celui-ci s'est déroulé ?\n \nCette analyse des incidents en fonction des Etats sera complétée par une visualisation cartographique dans la partie suivante. \nIntéressons nous d'abord aux nombre de victimes des incidents par arme à feu dans chaque Etat.")

In [ ]:
grouped_data10 = base_totale.groupby('state').agg({
    'n_injured': 'sum',
    'n_killed': 'sum'
}).reset_index()


fig10 = px.bar(
    grouped_data10,
    x=grouped_data10['state'],
    y=['n_injured', 'n_killed'],
    labels={"state": "Etat", "value": "Nombre de victimes par incident", "variable": "Type de victime"},
    title="Nombre de victimes d'incident par arme à feu par Etat",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig10.update_layout(barmode='group')  # Display bars side by side

fig10.show()

In [ ]:
print("Ce graphique mets en évidence les Etats dans lesquels les victimes sont les plus nombreuses. Les Etats dans lesquels il y a eu le plus de blessés sont l'Illinois, le Maryland, la Virginie et le Missouri alors que ceux dans lesquels il y a eu le plus de morts sont la Californie, le Missouri, le Maryland, la Virginie et le Texas. \nOn voit déjà ici que selon l'endroit où un incident par arme à feu se déroule, il n'a pas les mêmes conséquences. Il y a donc des caractéristiques spécifiques aux Etats qui doivent expliquer ces différences. \nOn peut également remarquer que ces Etats ne sont pas exactement les mêmes que ceux dans lesquels il y a le plus d'incidents par arme à feu, un incident ne semble donc pas avoir la même gravité en termes de conséquences partout.")

In [ ]:
grouped_data11 = base_totale.groupby('state').agg({
    'n_injured': 'mean',
    'n_killed': 'mean'
}).reset_index()


fig11 = px.bar(
    grouped_data11,
    x=grouped_data11['state'],
    y=['n_injured', 'n_killed'],
    labels={"state": "Etat", "value": "Moyenne du nombre de victimes par incident", "variable": "Type de victime"},
    title="Nombre moyen de victimes par incident par arme à feu par Etat",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig11.update_layout(barmode='group')  # Display bars side by side

fig11.show()

In [ ]:
print("Même si les disparités entre Etats se sont atténuées par rapport au graphique précédent, ce graphique nous permet tout de même de mettre en évidence des différences en terme de nombre moyen de victimes par incident. \nAinsi, dans l'Illinois, les incidents font en moyenne plus de blessés que dans les autres Etats (presque 0,8 par incident) alors qu'ils font relativement peu de morts (0,18). \nAu contraire, dans certains Etats tels que l'Arizona, les incidents font un nombre moyen de blessés (0.46) mais la moyenne de morts par incident est très élevée (quasiment 0,47 par incident). \nLes taux de mortalité et de 'blessure' d'un incident varient donc beaucoup d'un Etat à l'autre, et ne sont pas nécessairement corrélés. Dans certains Etats, les incidents font peu de morts et peu de blessés (Maine, New Hampshire), \ndans d'autres ils font beaucoup de blessés et de morts (Arizona), dans d'autres encore ils font beaucoup de blessés et peu de morts (Illinois, Maryland) et finalement ils peuvent même faire plus de morts que de blessés en moyenne (Wyoming, Montana, Idaho).")

In [ ]:
grouped_data12 = base_totale.groupby('state').agg({
    'n_injured': 'sum',
    'n_killed': 'sum',
    'pop_state_2015':'first'
}).reset_index()
grouped_data12['injured_per_100K']=grouped_data12["n_injured"]/grouped_data12['pop_state_2015']*100
grouped_data12['killed_per_100K']=grouped_data12["n_killed"]/grouped_data12['pop_state_2015']*100

fig12 = px.bar(
    grouped_data12,
    x=grouped_data12['state'],
    y=['injured_per_100K', 'killed_per_100K'],
    labels={"state": "Etat", "value": "nombre de victimes pour 100 000 habitants", "variable": "Type de victime"},
    title="Nombre de victimes d'incident par arme à feu pour 100 000 habitants par Etat",
    color_discrete_map={"injured_per_100K": "firebrick", "killed_per_100K": "steelblue"},
)

fig12.update_layout(barmode='group')  # Display bars side by side

fig12.show()

In [ ]:
print("Ce graphique mets en évidence les Etats ayant le plus de blessés et de tués lors d'incidents par arme à feu pour 100 000 habitants. \nL'Etat District of Columbia est ici un outlier, alors qu'il ne l'était pas dans les autres mesures des conséquences des incidents, ce qui pourrait indiquer une erreur ou simplement une très forte concentration de victimes en comparaison à la taille de l'Etat. \nOn retrouve les Etats du Maryland, Missouri, Virginie et Illinois comme ceux dans lesquels les incidents par arme à feu font le plus de blessés et de morts. \nEn dehors de ces 11 Etats dans lesquels les incidents par arme à feu ont fait plus de 50 blessés pour 100 000  habitants entre 2013 et 2018, les autres sont tous relativement similaires et les incidents qui s'y déroulent ont provoqué entre 5 et 35 blessés pour 100 000 habitants.")

In [ ]:
grouped_data13 = base_totale.groupby('state').agg({
    'n_killed': 'mean',
    'indic_suicide':'mean'
}).reset_index()


fig13 = px.bar(
    grouped_data13,
    x=grouped_data13['state'],
    y=['n_killed', 'indic_suicide'],
    labels={"state": "Etat", "value": "Moyenne du nb de morts par incident, taux de suicide", "variable": "Décès"},
    title="Comparaison du nombre moyen de morts par incident et du taux de suicide parmi les incidents, par Etat",
    color_discrete_map={"n_killed": "firebrick", "indic_suicide": "steelblue"},
)

fig13.update_layout(barmode='group')  # Display bars side by side

fig13.show()

In [ ]:
print(f"La proportion de suicide parmi les incidents varie largement d'un Etat à l'autre, probablement en raison de caractéristiques socio-démographiques et économiques de ces Etats. On voit que cette proportion est la plus élevée dans l'Arizona (0.28), le Nevada, le Texas, l'Utah et l'Idaho. \nCette proportion est fortement corrélée au nombre de morts ({base_totale['indic_suicide'].corr(base_totale['n_killed'])}), ce qui est logique car la réalisation d'un suicide implique au moins un mort, mais elle est également négativement corrélée au nombre de blessés ({base_totale['indic_suicide'].corr(base_totale['n_injured'])}), \nce qui pourrait expliquer, en partie, pourquoi ces Etats précédemment mentionnés font partie de ceux dans lesquels le nombre de blessés et le nombre de morts par arme à feu sont relativement proches, au contraires d'Etats dans lesquels le nombre de blessés est largement supérieur à celui des morts.")